In [35]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict,OrderedDict
from string import punctuation
from heapq import nlargest
import nltk
from bs4 import BeautifulSoup
from nltk.tag import StanfordNERTagger
import requests
import operator

In [36]:
stanford_ner_dir = '/home/dell/stanford-ner-2015-04-20/'
eng_model_filename= stanford_ner_dir + 'classifiers/english.all.3class.distsim.crf.ser.gz'
my_path_to_jar= stanford_ner_dir + 'stanford-ner.jar'

st = StanfordNERTagger(model_filename=eng_model_filename, path_to_jar=my_path_to_jar)

In [42]:
class FrequencySummarizer:
    def __init__(self, min_cut=0.1, max_cut=0.9):
        """
         Initilize the text summarizer.
         Words that have a frequency term lower than min_cut 
         or higer than max_cut will be ignored.
        """
        self._min_cut = min_cut
        self._max_cut = max_cut 
        self._stopwords = set(stopwords.words('english') + list(punctuation))

    def _compute_frequencies(self, word_sent):
        """ 
          Compute the frequency of each of word.
          Input: 
           word_sent, a list of sentences already tokenized.
          Output: 
           freq, a dictionary where freq[w] is the frequency of w.
        """
        freq = defaultdict(int)
        for s in word_sent:
            for word in s:
                if word not in self._stopwords:
                    freq[word] += 1
    # frequencies normalization and fitering
        m = float(max(freq.values()))
        for w in freq.keys():
            freq[w] = freq[w]/m
            if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
                del freq[w]
        return freq

    def summarize(self, text, n):
        """
          Return a list of n sentences 
          which represent the summary of text.
        """
        sents = sent_tokenize(text)
        assert n <= len(sents)
        word_sent = [word_tokenize(s.lower()) for s in sents]
        self._freq = self._compute_frequencies(word_sent)
        ranking = defaultdict(int)
        for i,sent in enumerate(word_sent):
            for w in sent:
                if w in self._freq:
                    ranking[i] += self._freq[w]
        sents_idx = self._rank(ranking, n)    
        return [sents[j] for j in sents_idx]

    def _rank(self, ranking, n):
        """ return the first n sentences with highest ranking """
        return nlargest(n, ranking, key=ranking.get)

In [43]:
def getTextFromURL(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    return text

In [44]:
def compute_frequencies(tokenized_text,term):
    freq = 0
    for word in tokenized_text:
        if word == term:
            freq+=1
    return freq

In [45]:
'''
def summarizeURL(url, total_pars):
    url_text = getTextFromURL(url).replace(u"Â", u"").replace(u"â", u"")

    fs = FrequencySummarizer()
    final_summary = fs.summarize(url_text.replace("\n"," "), total_pars)
    return " ".join(final_summary)
'''
#url = raw_input("Enter a URL\n")
#stop_words = set(stopwords.words('english') + list(punctuation))
#print type(stopwords.words('english'))

url = 'http://www.theedadvocate.org/?p=2241'
url_text = getTextFromURL(url)
fs = FrequencySummarizer()
final_summary = fs.summarize(url_text.replace("\n"," "),7)
final_summary = " ".join(final_summary)
#sents = sent_tokenize(final_summary)
#word_sent = [word_tokenize(s.lower()) for s in sents]
#print word_sent[0]
tokenized_text = word_tokenize(final_summary)
#print tokenized_text[0]
freq = defaultdict(int)
StopWords = set(nltk.corpus.stopwords.words('english'))
classified_text = st.tag(tokenized_text)
#for term,tag in classified_text:
#    print term, tag
for term,tag in classified_text:
    if tag == 'LOCATION' and term not in StopWords:
        freq[term]=compute_frequencies(tokenized_text,term)
#final_summary = summarizeURL(url, 5)
#print final_summary


In [46]:
print freq.keys()
#for key,value in freq.iteritems():
#    print key,value

sorted_freq = OrderedDict(sorted(freq.items(), key=operator.itemgetter(1), reverse = True))
#print type(sorted_freq)
#for key,value in sorted_freq.iteritems():
#    print key,value
    
print sorted_freq.keys()[0]

[u'America']
America
